In [39]:
import pandas as pd
import numpy as np
from datetime import timedelta

In [40]:
# Tải dữ liệu
data = pd.read_csv(r"C:\Users\Admin\Downloads\FE_hourly.csv", parse_dates=['Datetime'])

In [41]:
# Kiểm tra giá trị thiếu
missing_values = data.isnull().sum()
print("Giá trị thiếu:\n", missing_values)

Giá trị thiếu:
 Datetime    0
FE_MW       0
dtype: int64


In [42]:
# Kiểm tra tính liên tục của thời gian
data['time_diff'] = data['Datetime'].diff()
time_gaps = data[data['time_diff'] != timedelta(hours=1)]
print("Các khoảng thời gian không liên tục:\n", time_gaps)

Các khoảng thời gian không liên tục:
                  Datetime   FE_MW         time_diff
0     2011-12-31 01:00:00  6222.0               NaT
24    2011-12-30 01:00:00  6601.0 -2 days +01:00:00
48    2011-12-29 01:00:00  7253.0 -2 days +01:00:00
72    2011-12-28 01:00:00  6942.0 -2 days +01:00:00
96    2011-12-27 01:00:00  6482.0 -2 days +01:00:00
...                   ...     ...               ...
62754 2018-01-05 01:00:00  9009.0 -2 days +01:00:00
62778 2018-01-04 01:00:00  8329.0 -2 days +01:00:00
62802 2018-01-03 01:00:00  8891.0 -2 days +01:00:00
62826 2018-01-02 01:00:00  8138.0 -2 days +01:00:00
62850 2018-01-01 01:00:00  7907.0 -2 days +01:00:00

[2634 rows x 3 columns]


In [43]:
# Xóa cột tạm
data = data.drop(columns=['time_diff'])

In [68]:
from sklearn.preprocessing import MinMaxScaler

# Chuẩn hóa dữ liệu
scaler = MinMaxScaler(feature_range=(0, 1))
scaled_data = scaler.fit_transform(data['FE_MW'].values.reshape(-1, 1))# Tạo đặc trưng từ thời gian
data['Hour'] = data['Datetime'].dt.hour
data['IsPeakHour'] = data['Hour'].isin([8, 9, 10, 17, 18, 19]).astype(int)  # Giờ cao điểm
data['DayOfWeek'] = data['Datetime'].dt.dayofweek
data['Month'] = data['Datetime'].dt.month
data['DayOfMonth'] = data['Datetime'].dt.day
data['Year'] = data['Datetime'].dt.year

In [69]:
# Tạo các giá trị lag
for lag in range(1, 49):  # Tăng lag lên 48 giờ để nắm bắt mẫu tốt hơn
    data[f'lag_{lag}'] = data['FE_MW'].shift(lag)

In [70]:
# Tạo trung bình động
data['MA_24'] = data['FE_MW'].rolling(window=24).mean()
data['MA_168'] = data['FE_MW'].rolling(window=168).mean()

In [71]:
# Loại bỏ các hàng có giá trị NaN do lag và rolling
data = data.dropna()

In [72]:
# Chia dữ liệu: Lấy 1 tháng cuối để huấn luyện và dự báo 24 giờ tiếp theo
train_end = data['Datetime'].max() - timedelta(hours=24)
train_data = data[data['Datetime'] <= train_end][-720:]  # 1 tháng (720 giờ)
test_data = data[data['Datetime'] > train_end][:24]  # 24 giờ tiếp theo

In [74]:
# Chuẩn bị đặc trưng và nhãn
features = ['Hour', 'IsPeakHour', 'DayOfWeek', 'Month', 'DayOfMonth', 'Year', 'MA_24', 'MA_168'] + [f'lag_{lag}' for lag in range(1, 49)]
X_train = train_data[features]
y_train = train_data['FE_MW']
X_test = test_data[features]
y_test = test_data['FE_MW']

In [75]:
# Xây dựng và huấn luyện mô hình XGBoost với tham số tinh chỉnh
xgb_model = xgb.XGBRegressor(
    n_estimators=200,  # Tăng số cây để học tốt hơn
    learning_rate=0.05,  # Giảm để học chậm và ổn định hơn
    max_depth=7,  # Tăng độ sâu để nắm bắt mẫu phức tạp
    min_child_weight=2,
    subsample=0.9,
    colsample_bytree=0.9,
    objective='reg:squarederror',
    random_state=42
)
xgb_model.fit(X_train, y_train)

# Dự báo
y_pred = xgb_model.predict(X_test)

In [76]:
from sklearn.metrics import mean_absolute_error, mean_squared_error

# Hàm tính toán các chỉ số đánh giá
def calculate_metrics(actual, predicted):
    mae = mean_absolute_error(actual, predicted)
    rmse = np.sqrt(mean_squared_error(actual, predicted))
    mape = np.mean(np.abs((actual - predicted) / actual)) * 100
    return mae, rmse, mape

In [77]:
# Đánh giá mô hình
xgb_mae, xgb_rmse, xgb_mape = calculate_metrics(y_test, y_pred)
print(f"XGBoost - MAE: {xgb_mae:.2f}, RMSE: {xgb_rmse:.2f}, MAPE: {xgb_mape:.2f}%")

XGBoost - MAE: 459.20, RMSE: 543.54, MAPE: 5.72%
